In [1]:
import data, models, vizz

In [2]:
# temp code for dev
import importlib
importlib.reload(data)
importlib.reload(models)
importlib.reload(vizz)

<module 'vizz' from '/Users/natalie/Documents/Google Drive/Code/kaggle/kaggle-disaster-tweets/src/vizz.py'>

In [3]:
# test = data.get_data('data/raw/train.csv')
from datasets import Dataset
train = data.get_data('../data/raw/train.csv').fillna('')
train['input'] = train['keyword'] + '; ' + train['location'] + "; " + train['text']
train_df, test_df = data.split_traning(train[['input','target']])
train_df.columns = ['text', 'label']
test_df.columns = ['text', 'label']
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

In [6]:
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
roberta_tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def preprocess_function(ds):
   return roberta_tokenizer(ds["text"], truncation=True)
 
roberta_tokenized_train = train_ds.map(preprocess_function, batched=True)
roberta_tokenized_test = test_ds.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=roberta_tokenizer)
roberta_model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

Map:   0%|          | 0/5709 [00:00<?, ? examples/s]

Map:   0%|          | 0/1904 [00:00<?, ? examples/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should pr

In [10]:
import numpy as np
from datasets import load_metric
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

repo_name = "nbrooks7/kaggle-dissaster-roberta"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
   use_mps_device=True,
)
 
trainer = Trainer(
   model=roberta_model,
   args=training_args,
   train_dataset=roberta_tokenized_train,
   eval_dataset=roberta_tokenized_test,
   tokenizer=roberta_tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

/Users/natalie/Documents/Google Drive/Code/kaggle/kaggle-disaster-tweets/src/nbrooks7/kaggle-dissaster-roberta is already a clone of https://huggingface.co/nbrooks7/kaggle-dissaster-roberta. Make sure you pull the latest changes with `repo.git_pull()`.


In [11]:
trainer.train()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/714 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.4124, 'learning_rate': 5.994397759103642e-06, 'epoch': 1.4}


Several commits (2) will be pushed upstream.


{'train_runtime': 549.3812, 'train_samples_per_second': 20.783, 'train_steps_per_second': 1.3, 'train_loss': 0.3901066927014947, 'epoch': 2.0}


TrainOutput(global_step=714, training_loss=0.3901066927014947, metrics={'train_runtime': 549.3812, 'train_samples_per_second': 20.783, 'train_steps_per_second': 1.3, 'train_loss': 0.3901066927014947, 'epoch': 2.0})

In [12]:
trainer.evaluate()

  0%|          | 0/119 [00:00<?, ?it/s]

/var/folders/34/80_r0pms4szcmf68qwsg2lww0000gn/T/ipykernel_5360/1780478218.py:8: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'eval_loss': 0.4065452516078949,
 'eval_accuracy': 0.8382352941176471,
 'eval_f1': 0.7994791666666667,
 'eval_runtime': 38.1554,
 'eval_samples_per_second': 49.901,
 'eval_steps_per_second': 3.119,
 'epoch': 2.0}

In [5]:
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer

roberta_large_tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")

def rob_large_preprocess_function(ds):
   return roberta_large_tokenizer(ds["text"], truncation=True)
 
roberta_large_tokenized_train = train_ds.map(rob_large_preprocess_function, batched=True)
roberta_large_tokenized_test = test_ds.map(rob_large_preprocess_function, batched=True)

rob_large_data_collator = DataCollatorWithPadding(tokenizer=roberta_large_tokenizer)
roberta_large_model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli", num_labels=2, ignore_mismatched_sizes=True)

Map:   0%|          | 0/5709 [00:00<?, ? examples/s]

Map:   0%|          | 0/1904 [00:00<?, ? examples/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large-mnli and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.out_

In [6]:
import numpy as np
from datasets import load_metric
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

local_repo_name = '/Users/natalie/Documents/Google Drive/Code/kaggle/kaggle-disaster-tweets/models'
training_args_large = TrainingArguments(
   output_dir=local_repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   use_mps_device=True,
)
 
trainer_large = Trainer(
   model=roberta_large_model,
   args=training_args_large,
   train_dataset=roberta_large_tokenized_train,
   eval_dataset=roberta_large_tokenized_test,
   tokenizer=roberta_large_tokenizer,
   data_collator=rob_large_data_collator,
   compute_metrics=compute_metrics,
)

In [7]:
trainer_large.train()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/714 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.4304, 'learning_rate': 5.994397759103642e-06, 'epoch': 1.4}
{'train_runtime': 3108.1396, 'train_samples_per_second': 3.674, 'train_steps_per_second': 0.23, 'train_loss': 0.40284555899996716, 'epoch': 2.0}


TrainOutput(global_step=714, training_loss=0.40284555899996716, metrics={'train_runtime': 3108.1396, 'train_samples_per_second': 3.674, 'train_steps_per_second': 0.23, 'train_loss': 0.40284555899996716, 'epoch': 2.0})

In [8]:
trainer_large.evaluate()

  0%|          | 0/119 [00:00<?, ?it/s]

/var/folders/34/80_r0pms4szcmf68qwsg2lww0000gn/T/ipykernel_6164/1786490856.py:8: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'eval_loss': 0.4467601478099823,
 'eval_accuracy': 0.8366596638655462,
 'eval_f1': 0.7947194719471947,
 'eval_runtime': 69.4115,
 'eval_samples_per_second': 27.431,
 'eval_steps_per_second': 1.714,
 'epoch': 2.0}